In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math

In [3]:
def calculate_atm_params(cruise_alt):
    isa = pd.read_csv("isa.csv")
    diff = cruise_alt % 500
    h_floor = math.floor(cruise_alt/500)*500
    h_ceil = math.ceil(cruise_alt/500)*500
    prop_floor = np.array([isa[isa['Elevation'] == h_floor].Temperature, isa[isa['Elevation'] == h_floor].Pressure, isa[isa['Elevation'] == h_floor]['Relative Density']])
    prop_ceil = np.array([isa[isa['Elevation'] == h_ceil].Temperature, isa[isa['Elevation'] == h_ceil].Pressure, isa[isa['Elevation'] == h_ceil]['Relative Density']])
    prop = prop_floor + (prop_ceil - prop_floor)*(diff/500)
    prop[1] *= 10e5
    prop[2] *= 1.2985
    return prop

The normal cruise speed of the F-22 Raptor can vary depending on the specific mission and operating conditions, but it is typically around Mach 1.5 (1,150 mph or 1,850 km/h) at high altitude.

The F-22 is designed for air superiority missions, and it is capable of flying at high speeds while maintaining maneuverability and stealth. Its maximum speed is classified, but it is believed to be around Mach 2.25 (1,500 mph or 2,410 km/h) at high altitude.

The normal cruise altitude of the F-22 Raptor is around 40,000 to 50,000 feet (12,000 to 15,000 meters) instead of 50,000 to 60,000 feet.


Sources:
The official website of the U.S. Air Force states that the F-22 has a "service ceiling" of "over 50,000 feet" and a "normal operating altitude" of "between 30,000 and 50,000 feet."
(Source: https://www.af.mil/About-Us/Fact-Sheets/Display/Article/104507/f-22-raptor/)
The official website of the U.S. Air Force states that the F-22 has a "maximum speed greater than Mach 2.25" and a "cruise speed greater than Mach 1.5."
(Source: https://www.af.mil/About-Us/Fact-Sheets/Display/Article/104507/f-22-raptor/)

In [4]:
R = 287.1
gamma = 1.4
gamma_g = 4/3
c_p = R*(gamma/(gamma-1))
c_p_g = R*(gamma_g/(gamma_g-1))
cruise_alt = 13500
cruise_mach_no = 1.5
alpha = 0.3
T0, p0, rho0 = calculate_atm_params(cruise_alt)
v0 = 1.5*np.sqrt(gamma*R*T0)
M0 = cruise_mach_no

Pressure Recovery Factor(PRF) = 0.95 to 0.99

Adiabatic efficiency = 90-95%

Source:

1. "Jet Propulsion: A Simple Guide to the Aerodynamics and Thermodynamic Design and Performance of Jet Engines" by Nicholas Cumpsty and Andrew Heyes (2020): This textbook also provides an overview of gas turbine engines and their components, including inlets. According to the book, the typical PRF of modern aircraft inlets is around 0.95 to 0.99, while the typical adiabatic efficiency of the inlet ranges from 90% to 95%.

In [65]:
inlet_diameter = 1.17
r = inlet_diameter/2
m_dot = rho0*v0*np.pi*r*r
m_dot_core = m_dot/(1 + alpha)
m_dot_bypass = m_dot - m_dot_core
print(f"Mass Flow Rate: ", "{0:.3f}".format(m_dot[0]), "kg/s")

Mass Flow Rate:  124.346 kg/s


In [6]:
n_d = 0.9
prf = 0.95

In [7]:
def inlet(M0, p0, T0, n_d, prf):
    p_t0 = p0*((1 + ((gamma-1)/2)*M0**2)**(gamma/(gamma-1)))
    p_t1 = 0.95*p_t0
    pi_d = p_t1/p0
    tau_r = 1 + (((pi_d**((gamma-1)/gamma)) - 1)/n_d)
    T_t0 = tau_r*T0
    T_t1 = T_t0
    return p_t1, T_t1, pi_d

In [67]:
p_t1, T_t1, pi_d = inlet(M0, p0, T0, n_d, prf)
print(f"Compressor Inlet Total Pressure:", "{0:.3f}".format(p_t1[0]/10e5), "bar")
print(f"Compressor Inlet Total Temperature:", "{0:.3f}".format(T_t1[0]), "K")

Compressor Inlet Total Pressure: 0.535 bar
Compressor Inlet Total Temperature: 319.971 K


In [68]:
def compressor(p_t1, T_t1, n_c, pi_c):
    tau_c = 1 + (((pi_c)**((gamma-1)/gamma) - 1)/n_c)
    p_t2 = pi_c*p_t1
    T_t2 = tau_c*T_t1
    return p_t2, T_t2

Low Pressure Compressor

In [69]:
n_c_l = 0.85
pi_c_l = 3

In [73]:
p_t2, T_t2 = compressor(p_t1, T_t1, n_c_l, pi_c_l)
print(f"High Pressure Compressor Inlet Total Pressure:", "{0:.3f}".format(p_t2[0]/10e5), "bar")
print(f"High Pressure Compressor Inlet Total Temperature:", "{0:.3f}".format(T_t2[0]), "K")

High Pressure Compressor Inlet Total Pressure: 1.604 bar
High Pressure Compressor Inlet Total Temperature: 458.777 K


High Pressure Compressor

In [74]:
n_c_h = 0.9
pi_c_h = 25/3

In [77]:
p_t3, T_t3 = compressor(p_t2, T_t2, n_c_h, pi_c_h)
print(f"Burner Inlet Total Pressure:", "{0:.3f}".format(p_t3[0]/10e5), "bar")
print(f"Burner Inlet Total Temperature:", "{0:.3f}".format(T_t3[0]), "K")

Burner Inlet Total Pressure: 13.366 bar
Burner Inlet Total Temperature: 883.247 K


### Burner

In [78]:
T_max = 1922
Q_R = 43*10e6
n_b = 0.45
press_loss = 0.04

In [79]:
def burner(p_t0, T_t0, Q_R, n_b, T_max):
    m_f_dot = m_dot_core*(c_p_g*T_max - c_p*T_t0)/(n_b*Q_R - c_p_g*T_max)
    return m_f_dot

In [80]:
T_t4 = T_max
p_t4 = (1 - press_loss) * p_t3
m_f_dot = burner(p_t3, T_t3, Q_R, n_b, T_max)
print(f"Rate of fuel burning:", "{0:.3f}".format(m_f_dot[0]), "kg/s")
print(f"Fuel-to-Air ratio:", "{0:.3f}".format((m_f_dot[0]/m_dot_core[0])*100), "%")

Rate of fuel burning: 0.660 kg/s
Fuel-to-Air ratio: 0.690 %


### Turbine

In [81]:
n_t = 0.95

In [82]:
def turbine(p_t0, T_t0, n_t, T_c0, T_c1):
    T_t1 = T_t0 - m_dot*((c_p*(T_c1 - T_c0))/(n_t*c_p_g*(m_dot + m_f_dot)))
    pi_t = ((1 - (((T_t0 - T_t1)*n_t))/T_t0))**(gamma_g/(gamma_g-1))
    p_t1 = p_t0*pi_t
    return p_t1, T_t1

In [83]:
p_t5, T_t5 = turbine(p_t4, T_t4, n_t, T_t2, T_t3)
print(f"Low Pressure Turbine Inlet Temperature:", "{0:.3f}".format(T_t5[0]), "K")
print(f"Low Pressure Turbine Inlet Pressure:", "{0:.3f}".format(p_t5[0]/10e5), "bar")

Low Pressure Turbine Inlet Temperature: 1533.104 K
Low Pressure Turbine Inlet Pressure: 5.463 bar


In [84]:
p_t6, T_t6 = turbine(p_t5, T_t5, n_t, T_t1, T_t2)
print(f"Low Pressure Turbine Exit Temperature:", "{0:.3f}".format(T_t6[0]), "K")
print(f"Low Pressure Turbine Exit Pressure:", "{0:.3f}".format(p_t6[0]/10e5), "bar")

Low Pressure Turbine Exit Temperature: 1405.931 K
Low Pressure Turbine Exit Pressure: 3.934 bar


### Nozzle

In [85]:
n_n = 0.9

In [86]:
def nozzle(p_t0, T_t0, n_n, p_a, gamma):
    NPR = p_t6[0]/p0[0]
    pi_n = ((NPR)**((gamma - 1)/gamma) - n_n*((NPR)**((gamma - 1)/gamma) - 1))**(-(gamma/(gamma - 1)))
    p_t1 = p_t0 * pi_n
    M = np.sqrt(((p_t1/p_a) - 1)*(2/(gamma - 1)))
    T_1 = T_t0/(1 + ((gamma - 1)/2)*M*M)
    a = np.sqrt(gamma*R*T_1)
    V = M*a
    return V

In [87]:
v_j = nozzle(p_t6, T_t6, n_n, p0, gamma_g)[0]
print(f"Exit jet velocity:", "{0:.3f}".format(v_j), "m/s")

Exit jet velocity: 1739.981 m/s


In [88]:
thrust_core = (m_dot_core + m_f_dot)*v_j - m_dot_core*v0
print(f"Thrust generated by core:", "{0:.3f}".format(thrust_core[0]/1e3), "kN")

Thrust generated by core: 125.235 kN


For bypass flow

In [89]:
v_b = nozzle(p_t2, T_t2, n_n, p0, gamma_g)[0]

In [90]:
thrust_bypass = m_dot_bypass*(v_b - v0)
print(f"Thrust generated by bypass:", "{0:.3f}".format(thrust_bypass[0]/1e3), "kN")

Thrust generated by bypass: 14.404 kN


In [91]:
thrust_total = thrust_bypass + thrust_core
print(f"Total Thrust generated by the engine:", "{0:.3f}".format(thrust_total[0]/1e3), "kN")

Total Thrust generated by the engine: 139.639 kN
